# Porto Seguro's Safe Driver Prediction

****This notebook was written using guidance from a really great tutorial by Kaggle user Anisotropic:
### [Interactive Porto Insights - A Plot.ly Tutorial](https://www.kaggle.com/arthurtok/interactive-porto-insights-a-plot-ly-tutorial)



My attempt with this notebook is to explore the interactive data visualization tools offered by plot.ly, and perform some optimization using any insights I can gain from the data.

In this project, Porto Seguro has provided driver characteristic data related to car insurance claims. The objective is as follows:

## Problem Description
Predict the probability that a driver will file a claim in the next year.

### Data Exploration

We first begin by loading in the necessary packages for the data analysis

In [ ]:
# for general handling and manipulation of table data
import pandas as pd
import numpy as np

# for visualization of missing data entries
import missingno as msno

# for generation of interactive data visualization
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

# for random forest model predictions and result analysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

# start a random seed for reproducibility of results
np.random.seed(1)

Next we will load the data and get a sense of the general structure of information available

In [ ]:
train_data = pd.read_csv('../input/train.csv')

In [ ]:
print('train_data({0[0]},{0[1]})'.format(train_data.shape))

print('Number of training examples: {0}'.format(train_data.shape[0]))
print('Number of features for each example: {0}'.format(train_data.shape[1]))

Below is a list of the labels for each of the 59 data features

In [ ]:
pd.DataFrame(data = {'Feature Label': train_data.columns})

The data has the following feature structure within its 59 features:

ID - identifier of the driver 

target - indicates whether or not a claim was filed for that driver

For the rest of the feature labels:
1. The "ps" prefix occurs before all features (presumably for "Porto Seguro")

2. Features are then grouped into the following categories

    a. 'ind' - contains 18 features (these features are relating to **ind**ividual)

    b. 'reg' - contains 3 features (these features are relating to **reg**ion)

    c. 'car' - contains 15 features (these features are relating to  **car**)

    d. 'calc' - contains 20 features (these are **calc**ulated features)

3. In addition, some features have post have the following suffixes:
    a. 'cat' - used to denote that the feature is categorical
    b. 'bin' - used to denote that the feature is binary
    
In order to keep things a little neater, we will drop the "ps" prefix from the feature labels.

In [ ]:
no_ps = [train_data.columns[x][3:] for x in range(2, len(train_data.columns))]
train_data.columns = train_data.columns[:2].tolist() + no_ps

We are also given the information from Porto Seguro that missing data in this dataset is marked by a '-1' value. Any columns with a '-1' value present are listed below along with the number of missing entries for each of these columns.

In [ ]:
NA_columns = train_data.columns[train_data.isin(['-1']).any()]
NA_data_counts = train_data.isin(['-1']).sum()[train_data.isin(['-1']).any()]
pd.DataFrame(data = NA_data_counts, columns = ['# of missing entries'])

We can see that a few columns such as 'reg_03', 'car_03_cat' and 'car_05_cat', among others, have a significant amount of missing data entries.  This is something that we will explore later on, but for now we will ignore the missing values and work with the original training data set provided.

We can also visualize the missing data in these columns by using the missingno package.

In [ ]:
NA_data = train_data[NA_columns].replace(-1, np.NaN)
msno.matrix(df=NA_data, color = (0, 0.3, 0.3))

### Visualization

Let's now begin to visualize different aspects of the data to see what we can learn about the information contained in the features we've been given.

Starting off we will look at the target variable, which is the main feature of interest we are asked to predict. The pie graph below shows the relative distribution of the target variable.

In [ ]:
labels = ["Target = 0", "Target = 1"]
values = train_data["target"].value_counts().values

trace = go.Pie(labels = labels, values = values)
layout = go.Layout(title = 'Distribution of Target Variable')

fig = go.Figure(data = [trace], layout = layout)
iplot(fig)

As we can see, the target variable is highly skewed, with only 3.64% of drivers in the training data filing a claim. This is something that we will have to keep in mind when evaluating the results of our predictions later on.

Next we will take a look at the binary variables within the dataset

In [ ]:
bin_columns = train_data.columns[train_data.columns.str.contains('_bin')]

print("# of binary features: {0}".format(len(bin_columns)))

We can use a stacked bar graph to effectively get a sense of the distribution of values within the binary features, shown below:

In [ ]:
bin_counts = train_data[bin_columns].apply(pd.value_counts)

trace = []
for i in range(bin_counts.shape[0]):
    trace_temp = go.Bar(
        x= np.asarray(bin_columns),
        y= bin_counts.values[i],
        name = bin_counts.index[i]
    )
    trace.append(trace_temp)

layout = go.Layout(
    barmode = 'stack',
    title = 'Distribution of Binary Features'
)

fig = go.Figure(data = trace, layout = layout)
iplot(fig)

We should also get some information on the categorical variables in this dataset. We can do this by subsetting the categorical feature columns.

In [ ]:
cat_columns = train_data.columns[train_data.columns.str.contains('_cat')]
cat_data = pd.DataFrame(data = {'# of levels': train_data[cat_columns].max()})

cat_data

Similar to our binary distribution, below is a visualization of the distribution within each categorical feature. The legend on the right shows the labels provided each category. Keep in mind that '-1' denotes missing data.

In [ ]:
cat_counts = train_data[cat_columns].apply(pd.value_counts)

trace = []
for i in range(cat_counts.shape[0]):
    trace_temp = go.Bar(
        x= np.asarray(cat_columns),
        y= cat_counts.values[i],
        name = cat_counts.index[i]
    )
    trace.append(trace_temp)

layout = go.Layout(
    barmode = 'stack',
    title = 'Distribution of Categorical Features'
)

fig = go.Figure(data = trace, layout = layout)
iplot(fig)

Finally, we will take a look at the remaining features

In [ ]:
#Isolate the columns that are not binary or categorical
misc_columns = train_data.columns.drop(cat_columns).drop(bin_columns).drop(["id", "target"])

#Split these columns by group
ind_columns = misc_columns[misc_columns.str.contains('ind')]
reg_columns = misc_columns[misc_columns.str.contains('reg')]
car_columns = misc_columns[misc_columns.str.contains('car')]
calc_columns = misc_columns[misc_columns.str.contains('calc')]

#create boxplots for 'ind' columns
trace1 = []
for i in range(len(ind_columns)):
    trace_temp = go.Box(
        y= np.random.choice(train_data[ind_columns[i]], 2000, replace=False),
        name = ind_columns[i]
    )

    trace1.append(trace_temp)

layout1 = go.Layout(
    title = 'Distribution of "ind" Features'
)

# create boxplots for 'reg' columns
trace2 = []
for i in range(len(reg_columns)):
    trace_temp = go.Box(
        y= np.random.choice(train_data[reg_columns[i]], 2000, replace=False),
        name = reg_columns[i],
        boxpoints = 'suspectedoutliers'
    )

    trace2.append(trace_temp)

layout2 = go.Layout(
    title = 'Distribution of "reg" Features'
)

# create boxplots for 'car' columns
trace3 = []
for i in range(len(car_columns)):
    trace_temp = go.Box(
        y= np.random.choice(train_data[car_columns[i]], 2000, replace=False),
        name = car_columns[i],
        boxpoints = 'suspectedoutliers',
    )

    trace3.append(trace_temp)

layout3 = go.Layout(
    title = 'Distribution of "car" Features'
)

# create boxplots for 'calc' columns
trace4 = []
for i in range(len(calc_columns)):
    trace_temp = go.Box(
        y= np.random.choice(train_data[calc_columns[i]], 2000, replace=False),
        name = calc_columns[i],
        boxpoints = 'suspectectedoutliers'
    )

    trace4.append(trace_temp)

layout4 = go.Layout(
    title = 'Distribution of "calc" Features'
)

fig1 = go.Figure(data = trace1, layout = layout1)
fig2 = go.Figure(data = trace2, layout = layout2)
fig3 = go.Figure(data = trace3, layout = layout3)
fig4 = go.Figure(data = trace4, layout = layout4)


iplot(fig1)
iplot(fig2)
iplot(fig3)
iplot(fig4)

We can see from the distributions above that while some features have relatively small ranges (e.g. between 0 and 1) while others have relatively larger ranges, such as ps_calc_10 and ps_ind_15, it may be useful to consider normalization in order to improve the performance of our chosen model when it comes time to optimize our predictions.

Note that a random sample of 2000 drivers was selected for the box plots, since the load is pretty heavy on my machine, but it can be adapted for a larger subset.

For now, we will work with the data as-is and begin fitting the data to a Random Forest (RF) model in order to get a baseline performance that we will later attempt to optimize.

In [ ]:
# Separate rows where target is 0 or 1
target_data = [train_data["target"] == 0, train_data["target"] == 1]


trace = []
trace_temp = go.Box(
    y= np.random.choice(train_data['car_13'][target_data[0]],
                        2000,
                        replace=False
                       ),
    name = 'Target = 0',
    boxpoints = 'all',
    boxmean = 'sd'
)

trace.append(trace_temp)

trace_temp = go.Box(
    y= np.random.choice(train_data['car_13'][target_data[1]],
                        2000,
                        replace=False
                       ),
    name = 'Target = 1',
    boxpoints = 'all',
    boxmean = 'sd'
)

trace.append(trace_temp)

layout = go.Layout(
    title = 'car_13 Feature Distribution',
    width = 900,
    height = 1000,
)

fig = go.Figure(data = trace, layout = layout)
iplot(fig)

## Data Pre-processing

Let's begin preparing our data to build our predictive models. To start, we will split the training data into training ("train") and cross-validation ("CV") sets. The data will be split with the following distribution:

    90% train
    10% CV

In [ ]:
# Randomly select 10% of data for CV set
CV_index = np.random.choice(train_data.shape[0], int(train_data.shape[0]*.1), replace = False)
CV = train_data.iloc[CV_index, :]

# Use remaining 90% for model training data
train = train_data.drop(CV_index, axis = 0)

assert(train_data.shape[0] == train.shape[0]+CV.shape[0])

print('Number of training examples: {0}'.format(train.shape[0]))
print('Number of cross-validation examples: {0}'.format(CV.shape[0]))

Before we get started, we want to make sure that both our training and cross-validation sets contain a distribution of 0's and 1's for the target data

In [ ]:
labels = ["Target = 0", "Target = 1"]
train_values = train["target"].value_counts().values
CV_values = CV["target"].value_counts().values

#Distribution of training data set Target variable
trace1 = go.Pie(labels =  labels,
                values = train_values,
                domain= {"x": [0, 0.45]},
                hole = 0.3
               )

#Distribution of CV set Target variable
trace2 = go.Pie(labels = labels,
                values = CV_values,
                domain= {"x": [0.55, 1]},
                hole = 0.3
               )

layout = go.Layout(title = 'Distribution of Target Variable',
                    annotations = [{"text": "Train",
                                    "font": {"size": 20},
                                    "showarrow": False,
                                    "x": 0.19,
                                    "y": 0.5
                                   },
                                   {"text": "CV",
                                    "font": {"size": 20},
                                    "showarrow": False,
                                    "x": 0.8,
                                    "y": 0.5
                                   },
                                  ]
                   )

fig = go.Figure(data = [trace1, trace2], layout = layout)

iplot(fig)

Everything looks ok, so it's time to setup our model and make some predictions!

In [ ]:
# Set Random Forest Model parameters
rf = RandomForestClassifier(n_estimators=150, max_depth=8, min_samples_leaf=4, max_features=0.2, n_jobs=-1, random_state=0)

#Fit the model to our training data
rf.fit(train.drop(["id", "target"], axis=1), train.target)
print('------training done-------')

Now that we have trained our random forest model we should take a look at the feature importances that the model decided accounts for the variance in the Target variable.

In [ ]:
# make a list of the data features
features = train.drop(["id", "target"],axis=1).columns.values

# Extract feature importances
importance_df = pd.DataFrame(
    data = {'features': features,
            'Importance': rf.feature_importances_
           }
).sort_values(by='Importance',
              ascending=True)

# Feature importance barplot
trace = go.Bar(
    x=importance_df.iloc[:, 0],
    y=importance_df.iloc[:, 1],
    marker=dict(color = importance_df.iloc[:, 0],
                colorscale = 'Viridis',
                reversescale = True
               ),
    name = 'Random Forest Feature Importance',
    orientation = 'h'
)

layout = go.Layout(title='Barplot of Feature Importances',
                   width = 900,
                   height = 2000,
                  )
fig = go.Figure(data=[trace], layout = layout)
iplot(fig)

In [ ]:
#Predict probability of filing a claim (Target = 0 or 1)
rf_proba = rf.predict_proba(CV.drop(["id", "target"], axis=1))

#Isolate column of predicted probabilities that Target = 1
rf_proba_0 = rf_proba[:, 1].reshape(rf_proba.shape[0],1)

For this project the metric we will use to evaluate the performance of our model is based on the gini coefficient. This coefficient will rank the drivers in order from most to least likely to file a claim, and will compare the ranking between our predictions and the actual sorting according to our CV data.

Calculation of gini coefficient. Adapted from code submitted on 
Allstate Claim Prediction Challenge discussion boards
[here](https://www.kaggle.com/c/ClaimPredictionChallenge/discussion/703)

In [ ]:
def gini(a, p):
    data = np.asarray(np.c_[a, p, np.arange(len(a))],
                      dtype=np.float
                     )
    data = data[
        np.lexsort((data[:,2], -1*data[:,1]))
    ]
    totalLosses = data[:,0].sum()
    giniSum = data[:,0].cumsum().sum() / totalLosses

    giniSum -= (len(a) + 1) / 2.
    return giniSum / len(a)

def gini_norm(a, p):
    return gini(a, p) / gini(a, a)

gini_coef = gini_norm(CV['target'], rf_proba_0)

print('Normalized Gini Coefficient: {0:0.3f}'.format(gini_coef))

With this, we are able to get a gini coefficient score of 0.256. It's better than random guessing, which should be around 0, but there is still a long way to go here.

Next we will perform some error analysis to see where our model is lacking...

### In Progress...